In [1]:
import os
import numpy as np
from PIL import Image, ImageTk
from io import BytesIO

import requests
from tkinter import Tk, Label, Canvas, Entry, Button, NW

from keras.models import Sequential, load_model
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from keras.preprocessing import image as image_utils


In [2]:
model = Sequential()

#------- input layer----

model.add(Conv2D(32, (3, 3), input_shape = (128, 128, 3), activation = "relu"))
model.add(MaxPooling2D(pool_size = (2, 2)))
# -----------------

#--------- hidden layer-------
# here 64 is for ascending order in hidden layer (..., 32, 64, 128, 512, ...)
# for descending order in hidden layer (..., 512, 128, 64, 32, ...)

model.add(Conv2D(64, (3, 3), activation = "relu"))
model.add(MaxPooling2D(pool_size = (2, 2)))

model.add(Conv2D(128, (3, 3), activation = "relu"))
model.add(MaxPooling2D(pool_size = (2, 2)))
#-------------------------


#--- create bridge-------
# (for connecting b2n hidden layer and output layer)

model.add(Flatten())
#------------------

#----- output layer-----------
# for avoiding problems we will not create output layer directly, So, create FFNN
# 128 should be the same or equal to avoid collision

model.add(Dense(128, activation = "relu")) 
model.add(Dense(3, activation = "softmax")) 

In [3]:
model.compile(optimizer= "adam",
             loss = "categorical_crossentropy",
             metrics = ["accuracy"])

In [4]:
train_datagen = ImageDataGenerator(rescale = 1.0/255,
                                  shear_range = 0.2, 
                                  zoom_range = 0.2,
                                  horizontal_flip = True)

In [5]:
test_datagen = ImageDataGenerator(rescale = 1.0/255)

In [6]:
training_datasets = train_datagen.flow_from_directory("dataset/training_set", target_size = (128, 128), batch_size = 32, class_mode = "categorical") # local directory
testing_datasets = test_datagen.flow_from_directory("dataset/test_set", target_size = (128, 128), batch_size = 32, class_mode = "categorical") 

Found 2401 images belonging to 3 classes.
Found 600 images belonging to 3 classes.


In [7]:
model.fit(training_datasets, steps_per_epoch= 800/32, epochs = 50, validation_data = testing_datasets, validation_steps = 200/32)

Epoch 1/50
25/25 [==============================] - 23s 905ms/step - loss: 1.2049 - accuracy: 0.3413 - val_loss: 1.0894 - val_accuracy: 0.3259
Epoch 2/50
25/25 [==============================] - 24s 937ms/step - loss: 1.0857 - accuracy: 0.4013 - val_loss: 1.0395 - val_accuracy: 0.5000
Epoch 3/50
25/25 [==============================] - 23s 919ms/step - loss: 1.0662 - accuracy: 0.4300 - val_loss: 1.0227 - val_accuracy: 0.4821
Epoch 4/50
25/25 [==============================] - 22s 867ms/step - loss: 1.0051 - accuracy: 0.5175 - val_loss: 1.0081 - val_accuracy: 0.5357
Epoch 5/50
25/25 [==============================] - 21s 845ms/step - loss: 0.9750 - accuracy: 0.5113 - val_loss: 0.9039 - val_accuracy: 0.5402
Epoch 6/50
25/25 [==============================] - 21s 844ms/step - loss: 0.9133 - accuracy: 0.5600 - val_loss: 1.0549 - val_accuracy: 0.4866
Epoch 7/50
25/25 [==============================] - 21s 853ms/step - loss: 0.9873 - accuracy: 0.5238 - val_loss: 0.8802 - val_accuracy: 0.5714

In [8]:
# save model
model.save("model/model.h5")
model.save_weights("model/model_weights.h5")

In [9]:
image_width, image_height = 128, 128
model_path = "model/model.h5"
model_weight_path = "model/model_weights.h5"

final_model = load_model(model_path)
final_model.load_weights(model_weight_path)